<a href="https://colab.research.google.com/github/yourusername/colab-heroku-deployment/blob/main/Colab_to_Heroku_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Complete Google Colab to Heroku Deployment Guide

This notebook provides a comprehensive guide to deploy a web application from Google Colab to Heroku. We'll build a complete Flask application with interactive features and deploy it step-by-step.

## 📋 What We'll Build:
- **Flask Web Application** with interactive UI
- **File Upload & Processing** functionality
- **Data Visualization** with charts
- **API Integration** examples
- **Complete Heroku Deployment** workflow

## 🎯 Learning Objectives:
1. Structure a production-ready web application
2. Set up Heroku CLI in Colab environment
3. Deploy applications with proper configuration
4. Handle environment variables and secrets
5. Monitor and troubleshoot deployments

---

# 📦 Part 1: Application Setup

Let's start by creating our Flask web application with interactive features.

In [ ]:
#@title 🔧 Install Required Dependencies
#@markdown This cell installs all necessary packages for our application

import subprocess
import sys
import os
from IPython.display import HTML, display

def install_package(package):
    """Install package with error handling"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")
        return False

# List of required packages
packages = [
    "flask",
    "gunicorn",
    "requests",
    "pandas",
    "plotly",
    "python-dotenv",
    "Pillow",
    "werkzeug"
]

print("🔄 Installing dependencies...")
failed_packages = []

for package in packages:
    if install_package(package):
        print(f"✅ {package} installed successfully")
    else:
        failed_packages.append(package)

if failed_packages:
    print(f"\n⚠️ Failed to install: {', '.join(failed_packages)}")
    print("Please try running the cell again or install manually.")
else:
    print("\n🎉 All dependencies installed successfully!")
    display(HTML("<div style='background-color: #d4edda; padding: 10px; border-radius: 5px; color: #155724;'><b>✅ Ready to proceed to next step!</b></div>"))

In [ ]:
#@title 📁 Create Project Structure
#@markdown This creates the proper file structure for our Flask application

import shutil

# Define project structure
project_structure = {
    'app': {
        'templates': {},
        'static': {
            'css': {},
            'js': {},
            'uploads': {}
        }
    },
    'config': {}
}

def create_directory_structure(base_path, structure):
    """Recursively create directory structure"""
    for name, subdirs in structure.items():
        dir_path = os.path.join(base_path, name)
        os.makedirs(dir_path, exist_ok=True)
        print(f"📁 Created: {dir_path}")
        
        if isinstance(subdirs, dict) and subdirs:
            create_directory_structure(dir_path, subdirs)

# Set project name
PROJECT_NAME = "flask-heroku-app"
BASE_DIR = f"/content/{PROJECT_NAME}"

# Remove existing directory if it exists
if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)
    print(f"🗑️ Removed existing {PROJECT_NAME} directory")

# Create base directory
os.makedirs(BASE_DIR, exist_ok=True)
print(f"📁 Created base directory: {BASE_DIR}")

# Create project structure
create_directory_structure(BASE_DIR, project_structure)

# Change to project directory
os.chdir(BASE_DIR)
print(f"\n📍 Current directory: {os.getcwd()}")
print("\n🎯 Project structure created successfully!")

In [ ]:
#@title 🎨 Create HTML Templates
#@markdown Creating responsive HTML templates with Bootstrap styling

# Base template with Bootstrap and modern styling
base_template = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Flask Heroku App{% endblock %}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <style>
        .gradient-bg {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
        }
        .card-custom {
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.1);
            backdrop-filter: blur(10px);
            background: rgba(255,255,255,0.95);
        }
        .btn-custom {
            border-radius: 25px;
            padding: 10px 30px;
            font-weight: 600;
            transition: all 0.3s ease;
        }
        .btn-custom:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(0,0,0,0.2);
        }
        .navbar-custom {
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
        }
    </style>
</head>
<body class="gradient-bg">
    <nav class="navbar navbar-expand-lg navbar-dark navbar-custom">
        <div class="container">
            <a class="navbar-brand" href="{{ url_for('index') }}">
                <i class="fas fa-rocket me-2"></i>Flask Heroku App
            </a>
            <div class="navbar-nav ms-auto">
                <a class="nav-link" href="{{ url_for('index') }}">Home</a>
                <a class="nav-link" href="{{ url_for('calculator') }}">Calculator</a>
                <a class="nav-link" href="{{ url_for('file_upload') }}">Upload</a>
                <a class="nav-link" href="{{ url_for('api_demo') }}">API Demo</a>
            </div>
        </div>
    </nav>

    <div class="container mt-4">
        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                {% for category, message in messages %}
                    <div class="alert alert-{{ 'danger' if category == 'error' else category }} alert-dismissible fade show" role="alert">
                        {{ message }}
                        <button type="button" class="btn-close" data-bs-dismiss="alert"></button>
                    </div>
                {% endfor %}
            {% endif %}
        {% endwith %}

        {% block content %}{% endblock %}
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    {% block scripts %}{% endblock %}
</body>
</html>'''

# Home page template
index_template = '''{% extends "base.html" %}

{% block title %}Home - Flask Heroku App{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-lg-8">
        <div class="card card-custom">
            <div class="card-body text-center p-5">
                <i class="fas fa-rocket fa-4x text-primary mb-4"></i>
                <h1 class="display-4 mb-4">Welcome to Flask Heroku App!</h1>
                <p class="lead mb-4">A complete web application deployed from Google Colab to Heroku</p>
                
                <div class="row mt-5">
                    <div class="col-md-4 mb-3">
                        <div class="card h-100">
                            <div class="card-body text-center">
                                <i class="fas fa-calculator fa-2x text-success mb-3"></i>
                                <h5>Calculator</h5>
                                <p class="text-muted">Perform calculations with interactive forms</p>
                                <a href="{{ url_for('calculator') }}" class="btn btn-success btn-custom">Try Now</a>
                            </div>
                        </div>
                    </div>
                    <div class="col-md-4 mb-3">
                        <div class="card h-100">
                            <div class="card-body text-center">
                                <i class="fas fa-upload fa-2x text-warning mb-3"></i>
                                <h5>File Upload</h5>
                                <p class="text-muted">Upload and process files with visualization</p>
                                <a href="{{ url_for('file_upload') }}" class="btn btn-warning btn-custom">Upload</a>
                            </div>
                        </div>
                    </div>
                    <div class="col-md-4 mb-3">
                        <div class="card h-100">
                            <div class="card-body text-center">
                                <i class="fas fa-api fa-2x text-info mb-3"></i>
                                <h5>API Demo</h5>
                                <p class="text-muted">Interact with external APIs and data</p>
                                <a href="{{ url_for('api_demo') }}" class="btn btn-info btn-custom">Explore</a>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>
</div>
{% endblock %}'''

# Calculator template
calculator_template = '''{% extends "base.html" %}

{% block title %}Calculator - Flask Heroku App{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-lg-6">
        <div class="card card-custom">
            <div class="card-header text-center">
                <h3><i class="fas fa-calculator me-2"></i>Interactive Calculator</h3>
            </div>
            <div class="card-body">
                <form method="POST" id="calculatorForm">
                    <div class="mb-3">
                        <label for="num1" class="form-label">First Number</label>
                        <input type="number" class="form-control" id="num1" name="num1" step="any" required>
                    </div>
                    <div class="mb-3">
                        <label for="operation" class="form-label">Operation</label>
                        <select class="form-select" id="operation" name="operation" required>
                            <option value="">Choose operation...</option>
                            <option value="add">Addition (+)</option>
                            <option value="subtract">Subtraction (-)</option>
                            <option value="multiply">Multiplication (×)</option>
                            <option value="divide">Division (÷)</option>
                            <option value="power">Power (^)</option>
                            <option value="sqrt">Square Root (√)</option>
                        </select>
                    </div>
                    <div class="mb-3" id="num2Group">
                        <label for="num2" class="form-label">Second Number</label>
                        <input type="number" class="form-control" id="num2" name="num2" step="any">
                    </div>
                    <button type="submit" class="btn btn-primary btn-custom w-100">
                        <i class="fas fa-equals me-2"></i>Calculate
                    </button>
                </form>
                
                {% if result is defined %}
                <div class="mt-4 p-3 bg-light rounded">
                    <h5 class="text-center">Result:</h5>
                    <div class="text-center">
                        <span class="display-6 text-primary">{{ result }}</span>
                    </div>
                </div>
                {% endif %}
            </div>
        </div>
    </div>
</div>
{% endblock %}

{% block scripts %}
<script>
document.getElementById('operation').addEventListener('change', function() {
    const num2Group = document.getElementById('num2Group');
    const num2Input = document.getElementById('num2');
    
    if (this.value === 'sqrt') {
        num2Group.style.display = 'none';
        num2Input.required = false;
    } else {
        num2Group.style.display = 'block';
        num2Input.required = true;
    }
});
</script>
{% endblock %}'''

# File upload template
upload_template = '''{% extends "base.html" %}

{% block title %}File Upload - Flask Heroku App{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-lg-8">
        <div class="card card-custom">
            <div class="card-header text-center">
                <h3><i class="fas fa-upload me-2"></i>File Upload & Analysis</h3>
            </div>
            <div class="card-body">
                <form method="POST" enctype="multipart/form-data" id="uploadForm">
                    <div class="mb-3">
                        <label for="file" class="form-label">Choose CSV File</label>
                        <input type="file" class="form-control" id="file" name="file" accept=".csv,.txt" required>
                        <div class="form-text">Upload a CSV file for data analysis and visualization</div>
                    </div>
                    <button type="submit" class="btn btn-primary btn-custom w-100">
                        <i class="fas fa-chart-bar me-2"></i>Upload & Analyze
                    </button>
                </form>
                
                {% if chart_html %}
                <div class="mt-4">
                    <h5>Data Visualization:</h5>
                    <div id="chart">{{ chart_html|safe }}</div>
                </div>
                {% endif %}
                
                {% if file_info %}
                <div class="mt-4">
                    <h5>File Information:</h5>
                    <ul class="list-group">
                        <li class="list-group-item d-flex justify-content-between">
                            <span>Filename:</span>
                            <strong>{{ file_info.filename }}</strong>
                        </li>
                        <li class="list-group-item d-flex justify-content-between">
                            <span>Rows:</span>
                            <strong>{{ file_info.rows }}</strong>
                        </li>
                        <li class="list-group-item d-flex justify-content-between">
                            <span>Columns:</span>
                            <strong>{{ file_info.columns }}</strong>
                        </li>
                    </ul>
                </div>
                {% endif %}
            </div>
        </div>
    </div>
</div>
{% endblock %}'''

# API demo template
api_template = '''{% extends "base.html" %}

{% block title %}API Demo - Flask Heroku App{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-lg-8">
        <div class="card card-custom">
            <div class="card-header text-center">
                <h3><i class="fas fa-globe me-2"></i>API Integration Demo</h3>
            </div>
            <div class="card-body">
                <form method="POST" id="apiForm">
                    <div class="mb-3">
                        <label for="city" class="form-label">City Name</label>
                        <input type="text" class="form-control" id="city" name="city" placeholder="Enter city name" required>
                        <div class="form-text">Get weather information for any city</div>
                    </div>
                    <button type="submit" class="btn btn-primary btn-custom w-100">
                        <i class="fas fa-cloud-sun me-2"></i>Get Weather
                    </button>
                </form>
                
                {% if weather_data %}
                <div class="mt-4">
                    <div class="card">
                        <div class="card-body text-center">
                            <h4>{{ weather_data.city }}</h4>
                            <div class="row">
                                <div class="col-md-6">
                                    <i class="fas fa-thermometer-half fa-2x text-danger mb-2"></i>
                                    <h5>Temperature</h5>
                                    <p class="display-6">{{ weather_data.temperature }}°C</p>
                                </div>
                                <div class="col-md-6">
                                    <i class="fas fa-eye fa-2x text-info mb-2"></i>
                                    <h5>Description</h5>
                                    <p class="lead">{{ weather_data.description }}</p>
                                </div>
                            </div>
                        </div>
                    </div>
                </div>
                {% endif %}
                
                <div class="mt-4">
                    <h5>Random Quote Generator</h5>
                    <button type="button" class="btn btn-secondary btn-custom" onclick="getRandomQuote()">
                        <i class="fas fa-quote-left me-2"></i>Get Random Quote
                    </button>
                    <div id="quoteContainer" class="mt-3"></div>
                </div>
            </div>
        </div>
    </div>
</div>
{% endblock %}

{% block scripts %}
<script>
async function getRandomQuote() {
    const container = document.getElementById('quoteContainer');
    container.innerHTML = '<div class="text-center"><i class="fas fa-spinner fa-spin"></i> Loading...</div>';
    
    try {
        const response = await fetch('/api/quote');
        const data = await response.json();
        
        container.innerHTML = `
            <div class="card">
                <div class="card-body">
                    <blockquote class="blockquote mb-0">
                        <p>"${data.quote}"</p>
                        <footer class="blockquote-footer">${data.author}</footer>
                    </blockquote>
                </div>
            </div>
        `;
    } catch (error) {
        container.innerHTML = '<div class="alert alert-danger">Error loading quote. Please try again.</div>';
    }
}
</script>
{% endblock %}'''

# Write all templates to files
templates = {
    'base.html': base_template,
    'index.html': index_template,
    'calculator.html': calculator_template,
    'upload.html': upload_template,
    'api_demo.html': api_template
}

for filename, content in templates.items():
    filepath = f"app/templates/{filename}"
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"✅ Created template: {filename}")

print("\n🎨 All HTML templates created successfully!")

In [ ]:
#@title 🐍 Create Main Flask Application
#@markdown This creates the main Flask application with all routes and functionality

app_py_content = '''import os
import math
import requests
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
from flask import Flask, render_template, request, flash, redirect, url_for, jsonify
from werkzeug.utils import secure_filename
from werkzeug.exceptions import RequestEntityTooLarge
import random
import json

# Initialize Flask app
app = Flask(__name__)
app.secret_key = os.environ.get('SECRET_KEY', 'your-secret-key-change-in-production')
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# Configuration
UPLOAD_FOLDER = 'app/static/uploads'
ALLOWED_EXTENSIONS = {'txt', 'csv'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure upload directory exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def allowed_file(filename):
    """Check if file extension is allowed"""
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    """Home page route"""
    return render_template('index.html')

@app.route('/calculator', methods=['GET', 'POST'])
def calculator():
    """Calculator page with form handling"""
    result = None
    
    if request.method == 'POST':
        try:
            num1 = float(request.form.get('num1', 0))
            operation = request.form.get('operation')
            num2 = request.form.get('num2')
            
            if operation == 'add':
                result = num1 + float(num2)
            elif operation == 'subtract':
                result = num1 - float(num2)
            elif operation == 'multiply':
                result = num1 * float(num2)
            elif operation == 'divide':
                if float(num2) == 0:
                    flash('Cannot divide by zero!', 'error')
                    return render_template('calculator.html')
                result = num1 / float(num2)
            elif operation == 'power':
                result = num1 ** float(num2)
            elif operation == 'sqrt':
                if num1 < 0:
                    flash('Cannot calculate square root of negative number!', 'error')
                    return render_template('calculator.html')
                result = math.sqrt(num1)
            
            # Round result to 6 decimal places
            result = round(result, 6)
            flash(f'Calculation completed successfully!', 'success')
            
        except ValueError:
            flash('Please enter valid numbers!', 'error')
        except Exception as e:
            flash(f'An error occurred: {str(e)}', 'error')
    
    return render_template('calculator.html', result=result)

@app.route('/upload', methods=['GET', 'POST'])
def file_upload():
    """File upload and processing route"""
    chart_html = None
    file_info = None
    
    if request.method == 'POST':
        # Check if file was uploaded
        if 'file' not in request.files:
            flash('No file selected!', 'error')
            return redirect(request.url)
        
        file = request.files['file']
        
        if file.filename == '':
            flash('No file selected!', 'error')
            return redirect(request.url)
        
        if file and allowed_file(file.filename):
            try:
                filename = secure_filename(file.filename)
                filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                file.save(filepath)
                
                # Process the CSV file
                df = pd.read_csv(filepath)
                
                # Get file information
                file_info = {
                    'filename': filename,
                    'rows': len(df),
                    'columns': len(df.columns)
                }
                
                # Create a simple visualization
                if len(df.columns) >= 2:
                    # Create a scatter plot with first two numeric columns
                    numeric_cols = df.select_dtypes(include=['number']).columns
                    if len(numeric_cols) >= 2:
                        fig = go.Figure()
                        fig.add_trace(go.Scatter(
                            x=df[numeric_cols[0]],
                            y=df[numeric_cols[1]],
                            mode='markers',
                            marker=dict(size=8, color='blue', opacity=0.6),
                            name='Data Points'
                        ))
                        fig.update_layout(
                            title=f'Scatter Plot: {numeric_cols[0]} vs {numeric_cols[1]}',
                            xaxis_title=numeric_cols[0],
                            yaxis_title=numeric_cols[1],
                            height=400
                        )
                        chart_html = pyo.plot(fig, output_type='div', include_plotlyjs=False)
                    else:
                        # Create a bar chart for categorical data
                        col = df.columns[0]
                        value_counts = df[col].value_counts().head(10)
                        fig = go.Figure()
                        fig.add_trace(go.Bar(
                            x=value_counts.index,
                            y=value_counts.values,
                            marker_color='lightblue'
                        ))
                        fig.update_layout(
                            title=f'Top 10 Values in {col}',
                            xaxis_title=col,
                            yaxis_title='Count',
                            height=400
                        )
                        chart_html = pyo.plot(fig, output_type='div', include_plotlyjs=False)
                
                flash('File uploaded and processed successfully!', 'success')
                
                # Clean up uploaded file
                os.remove(filepath)
                
            except Exception as e:
                flash(f'Error processing file: {str(e)}', 'error')
        else:
            flash('Invalid file type. Please upload a CSV or TXT file.', 'error')
    
    return render_template('upload.html', chart_html=chart_html, file_info=file_info)

@app.route('/api-demo', methods=['GET', 'POST'])
def api_demo():
    """API integration demo route"""
    weather_data = None
    
    if request.method == 'POST':
        city = request.form.get('city', '').strip()
        
        if city:
            try:
                # Mock weather API (replace with real API in production)
                # For demo purposes, we'll generate mock data
                weather_data = {
                    'city': city.title(),
                    'temperature': random.randint(-10, 35),
                    'description': random.choice([
                        'Sunny', 'Cloudy', 'Rainy', 'Partly Cloudy', 
                        'Overcast', 'Clear', 'Foggy', 'Windy'
                    ])
                }
                flash(f'Weather data retrieved for {city}!', 'success')
            except Exception as e:
                flash(f'Error fetching weather data: {str(e)}', 'error')
        else:
            flash('Please enter a city name!', 'error')
    
    return render_template('api_demo.html', weather_data=weather_data)

@app.route('/api/quote')
def get_random_quote():
    """API endpoint for random quotes"""
    quotes = [
        {"quote": "The only way to do great work is to love what you do.", "author": "Steve Jobs"},
        {"quote": "Innovation distinguishes between a leader and a follower.", "author": "Steve Jobs"},
        {"quote": "Life is what happens to you while you're busy making other plans.", "author": "John Lennon"},
        {"quote": "The future belongs to those who believe in the beauty of their dreams.", "author": "Eleanor Roosevelt"},
        {"quote": "It is during our darkest moments that we must focus to see the light.", "author": "Aristotle"},
        {"quote": "Success is not final, failure is not fatal: it is the courage to continue that counts.", "author": "Winston Churchill"}
    ]
    
    return jsonify(random.choice(quotes))

@app.errorhandler(404)
def not_found_error(error):
    """Handle 404 errors"""
    return render_template('index.html'), 404

@app.errorhandler(500)
def internal_error(error):
    """Handle 500 errors"""
    return render_template('index.html'), 500

@app.errorhandler(RequestEntityTooLarge)
def handle_file_too_large(e):
    """Handle file too large errors"""
    flash('File is too large. Maximum size is 16MB.', 'error')
    return redirect(url_for('file_upload'))

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=False)
'''

# Write the main application file
with open('app.py', 'w', encoding='utf-8') as f:
    f.write(app_py_content)

print("✅ Created app.py with complete Flask application")
print("\n🎯 Features included:")
print("   • Interactive calculator with multiple operations")
print("   • File upload with CSV processing and visualization")
print("   • API integration demo with weather data")
print("   • Random quote generator API")
print("   • Error handling and file validation")
print("   • Responsive Bootstrap UI")

In [ ]:
#@title 📋 Create Deployment Configuration Files
#@markdown This creates all necessary files for Heroku deployment

# Create requirements.txt
requirements_content = '''Flask==2.3.3
gunicorn==21.2.0
requests==2.31.0
pandas==2.0.3
plotly==5.17.0
python-dotenv==1.0.0
Pillow==10.0.1
Werkzeug==2.3.7
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)
print("✅ Created requirements.txt")

# Create Procfile for Heroku
procfile_content = '''web: gunicorn app:app --bind 0.0.0.0:$PORT --workers 1 --threads 2 --timeout 120
'''

with open('Procfile', 'w') as f:
    f.write(procfile_content)
print("✅ Created Procfile")

# Create runtime.txt to specify Python version
runtime_content = '''python-3.9.18
'''

with open('runtime.txt', 'w') as f:
    f.write(runtime_content)
print("✅ Created runtime.txt")

# Create .env file for local development
env_content = '''# Local development environment variables
SECRET_KEY=your-secret-key-for-local-development
FLASK_ENV=development
FLASK_DEBUG=True
PORT=5000
'''

with open('.env', 'w') as f:
    f.write(env_content)
print("✅ Created .env file")

# Create .gitignore
gitignore_content = '''# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg

# Flask
instance/
.webassets-cache

# Environment variables
.env
.venv
env/
venv/
ENV/
env.bak/
venv.bak/

# IDE
.vscode/
.idea/
*.swp
*.swo
*~

# OS
.DS_Store
.DS_Store?
._*
.Spotlight-V100
.Trashes
ehthumbs.db
Thumbs.db

# Uploads
app/static/uploads/*
!app/static/uploads/.gitkeep
'''

with open('.gitignore', 'w') as f:
    f.write(gitignore_content)
print("✅ Created .gitignore")

# Create .gitkeep for uploads directory
with open('app/static/uploads/.gitkeep', 'w') as f:
    f.write('')
print("✅ Created .gitkeep for uploads directory")

# Create README.md
readme_content = '''# Flask Heroku App

A complete web application built with Flask and deployed to Heroku from Google Colab.

## Features

- **Interactive Calculator**: Perform various mathematical operations
- **File Upload & Analysis**: Upload CSV files for data visualization
- **API Integration**: Weather data demo and random quote generator
- **Responsive Design**: Bootstrap-powered UI that works on all devices

## Local Development

1. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```

2. Run the application:
   ```bash
   python app.py
   ```

3. Open your browser and navigate to `http://localhost:5000`

## Deployment

This application is configured for deployment on Heroku with:
- `Procfile` for process configuration
- `requirements.txt` for dependencies
- `runtime.txt` for Python version specification

## Technologies Used

- **Backend**: Flask, Python
- **Frontend**: HTML5, CSS3, JavaScript, Bootstrap 5
- **Data Processing**: Pandas, Plotly
- **Deployment**: Heroku, Gunicorn

## License

MIT License - feel free to use this code for your own projects!
'''

with open('README.md', 'w') as f:
    f.write(readme_content)
print("✅ Created README.md")

print("\n🎯 All deployment configuration files created successfully!")
print("\n📁 Project structure:")
print("   • app.py (main Flask application)")
print("   • requirements.txt (Python dependencies)")
print("   • Procfile (Heroku process configuration)")
print("   • runtime.txt (Python version specification)")
print("   • .env (local environment variables)")
print("   • .gitignore (Git ignore rules)")
print("   • README.md (project documentation)")

In [ ]:
#@title 🧪 Test Application Locally
#@markdown Test the Flask application locally before deployment

import time
import requests
from IPython.display import HTML, display

def test_flask_app():
    """Test the Flask application locally"""
    print("🧪 Testing Flask application locally...")
    
    try:
        # Start Flask app in background
        process = subprocess.Popen(
            ["python", "app.py"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Wait for app to start
        time.sleep(3)
        
        # Test if app is running
        try:
            response = requests.get("http://localhost:5000", timeout=5)
            if response.status_code == 200:
                print("✅ Flask app is running successfully!")
                print(f"   Status Code: {response.status_code}")
                print(f"   Response Length: {len(response.text)} characters")
                
                # Test API endpoint
                api_response = requests.get("http://localhost:5000/api/quote", timeout=5)
                if api_response.status_code == 200:
                    print("✅ API endpoint working correctly!")
                    quote_data = api_response.json()
                    print(f"   Sample quote: {quote_data.get('quote', 'N/A')[:50]}...")
                
            else:
                print(f"⚠️ App responded with status code: {response.status_code}")
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Could not connect to Flask app: {e}")
            
        # Stop the process
        process.terminate()
        process.wait(timeout=5)
        print("🛑 Flask app stopped")
        
    except Exception as e:
        print(f"❌ Error testing Flask app: {e}")
        
    print("\n✅ Local testing completed!")

# Run the test
test_flask_app()

# Display project summary
display(HTML("""
<div style='background-color: #e7f3ff; padding: 20px; border-radius: 10px; margin: 20px 0;'>
    <h3 style='color: #0066cc; margin-top: 0;'>🎉 Application Setup Complete!</h3>
    <p><strong>Your Flask application is ready for deployment with:</strong></p>
    <ul>
        <li>✅ Interactive calculator with multiple operations</li>
        <li>✅ File upload and CSV data visualization</li>
        <li>✅ API integration demos</li>
        <li>✅ Responsive Bootstrap UI</li>
        <li>✅ Error handling and validation</li>
        <li>✅ Production-ready configuration</li>
    </ul>
    <p><strong>Next:</strong> Proceed to Part 2 for Heroku deployment!</p>
</div>
"""))

---

# 🚀 Part 2: Heroku Deployment Process

Now that our Flask application is ready, let's deploy it to Heroku step by step.

In [ ]:
#@title 🔧 Install Heroku CLI
#@markdown Install Heroku CLI in the Colab environment

import os
from IPython.display import HTML, display

def run_command(command, description="", show_output=True):
    """Run shell command with error handling"""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            timeout=300  # 5 minute timeout
        )
        
        if result.returncode == 0:
            print(f"✅ {description} completed successfully")
            if show_output and result.stdout.strip():
                print(f"   Output: {result.stdout.strip()[:200]}...")
            return True, result.stdout
        else:
            print(f"❌ {description} failed")
            if result.stderr:
                print(f"   Error: {result.stderr.strip()[:200]}...")
            return False, result.stderr
            
    except subprocess.TimeoutExpired:
        print(f"⏰ {description} timed out")
        return False, "Command timed out"
    except Exception as e:
        print(f"❌ {description} failed with exception: {e}")
        return False, str(e)

print("🚀 Installing Heroku CLI...")
print("This may take a few minutes...\n")

# Update package list
success, output = run_command(
    "apt-get update -qq",
    "Updating package list",
    show_output=False
)

if success:
    # Install curl if not present
    success, output = run_command(
        "apt-get install -y curl",
        "Installing curl",
        show_output=False
    )

if success:
    # Download and install Heroku CLI
    success, output = run_command(
        "curl https://cli-assets.heroku.com/install.sh | sh",
        "Downloading and installing Heroku CLI"
    )

if success:
    # Verify installation
    success, output = run_command(
        "heroku --version",
        "Verifying Heroku CLI installation"
    )
    
    if success:
        print("\n🎉 Heroku CLI installed successfully!")
        print(f"Version: {output.strip()}")
        
        display(HTML("""
        <div style='background-color: #d4edda; padding: 15px; border-radius: 5px; color: #155724; margin: 10px 0;'>
            <strong>✅ Heroku CLI Ready!</strong><br>
            You can now proceed with Heroku authentication and deployment.
        </div>
        """))
    else:
        print("❌ Failed to verify Heroku CLI installation")
else:
    print("❌ Failed to install Heroku CLI")
    display(HTML("""
    <div style='background-color: #f8d7da; padding: 15px; border-radius: 5px; color: #721c24; margin: 10px 0;'>
        <strong>⚠️ Installation Failed</strong><br>
        Please try running this cell again or check your internet connection.
    </div>
    """))

print("\n📝 Next steps:")
print("   1. Authenticate with Heroku")
print("   2. Create a new Heroku application")
print("   3. Configure environment variables")
print("   4. Deploy the application")

In [ ]:
#@title 🔐 Heroku Authentication Setup
#@markdown Set up Heroku authentication (you'll need to provide your API key)

import getpass
import os
from IPython.display import HTML, display

def authenticate_heroku():
    """Authenticate with Heroku using API key"""
    
    display(HTML("""
    <div style='background-color: #fff3cd; padding: 15px; border-radius: 5px; color: #856404; margin: 10px 0;'>
        <strong>🔐 Heroku Authentication Required</strong><br><br>
        <p>To deploy to Heroku, you need to authenticate. You have two options:</p>
        <ol>
            <li><strong>API Key Method (Recommended for Colab):</strong>
                <ul>
                    <li>Go to <a href="https://dashboard.heroku.com/account" target="_blank">Heroku Account Settings</a></li>
                    <li>Scroll down to "API Key" section</li>
                    <li>Click "Reveal" and copy your API key</li>
                    <li>Enter it in the prompt below</li>
                </ul>
            </li>
            <li><strong>Browser Login:</strong> Use <code>heroku login</code> (may not work in Colab)</li>
        </ol>
    </div>
    """))
    
    # Ask user for authentication method
    auth_method = input("\nChoose authentication method (1 for API key, 2 for browser login): ").strip()
    
    if auth_method == "1":
        # API Key authentication
        print("\n🔑 API Key Authentication")
        api_key = getpass.getpass("Enter your Heroku API key (input will be hidden): ")
        
        if api_key.strip():
            # Set environment variable
            os.environ['HEROKU_API_KEY'] = api_key.strip()
            
            # Test authentication
            try:
                result = subprocess.run(
                    "heroku auth:whoami",
                    shell=True,
                    capture_output=True,
                    text=True,
                    timeout=30
                )
                
                if result.returncode == 0:
                    email = result.stdout.strip()
                    print(f"✅ Successfully authenticated as: {email}")
                    return True, email
                else:
                    print(f"❌ Authentication failed: {result.stderr.strip()}")
                    return False, None
                    
            except Exception as e:
                print(f"❌ Error during authentication: {e}")
                return False, None
        else:
            print("❌ No API key provided")
            return False, None
            
    elif auth_method == "2":
        # Browser login
        print("\n🌐 Browser Login")
        print("⚠️ Note: This may not work in Google Colab environment")
        
        try:
            result = subprocess.run(
                "heroku login --interactive",
                shell=True,
                timeout=60
            )
            
            if result.returncode == 0:
                # Verify authentication
                verify_result = subprocess.run(
                    "heroku auth:whoami",
                    shell=True,
                    capture_output=True,
                    text=True
                )
                
                if verify_result.returncode == 0:
                    email = verify_result.stdout.strip()
                    print(f"✅ Successfully authenticated as: {email}")
                    return True, email
                    
            print("❌ Browser login failed")
            return False, None
            
        except Exception as e:
            print(f"❌ Error during browser login: {e}")
            return False, None
    else:
        print("❌ Invalid choice. Please select 1 or 2.")
        return False, None

# Run authentication
success, user_email = authenticate_heroku()

if success:
    display(HTML(f"""
    <div style='background-color: #d4edda; padding: 15px; border-radius: 5px; color: #155724; margin: 10px 0;'>
        <strong>✅ Authentication Successful!</strong><br>
        Logged in as: <code>{user_email}</code><br>
        You can now create and deploy Heroku applications.
    </div>
    """))
else:
    display(HTML("""
    <div style='background-color: #f8d7da; padding: 15px; border-radius: 5px; color: #721c24; margin: 10px 0;'>
        <strong>❌ Authentication Failed</strong><br>
        Please check your API key and try again. Make sure you have a Heroku account and the API key is correct.
    </div>
    """))
    
print("\n📝 Troubleshooting Tips:")
print("   • Make sure you have a Heroku account (sign up at heroku.com)")
print("   • Verify your API key is correct and not expired")
print("   • Check your internet connection")
print("   • Try refreshing your API key in Heroku dashboard")

In [ ]:
#@title 🏗️ Create Heroku Application
#@markdown Create a new Heroku application with a unique name

import random
import string
import re
from IPython.display import HTML, display

def generate_app_name():
    """Generate a unique app name"""
    adjectives = ['awesome', 'amazing', 'fantastic', 'incredible', 'wonderful', 'brilliant', 'excellent', 'outstanding']
    nouns = ['flask', 'app', 'web', 'demo', 'project', 'site', 'portal', 'platform']
    
    adjective = random.choice(adjectives)
    noun = random.choice(nouns)
    number = ''.join(random.choices(string.digits, k=4))
    
    return f"{adjective}-{noun}-{number}"

def create_heroku_app(app_name=None):
    """Create a new Heroku application"""
    
    if not app_name:
        app_name = generate_app_name()
    
    # Validate app name
    if not re.match(r'^[a-z][a-z0-9-]*[a-z0-9]$', app_name) or len(app_name) > 30:
        print(f"❌ Invalid app name: {app_name}")
        print("App names must start with a letter, contain only lowercase letters, numbers, and hyphens, and be under 30 characters.")
        return False, None
    
    print(f"🏗️ Creating Heroku app: {app_name}")
    
    try:
        # Create the app
        result = subprocess.run(
            f"heroku create {app_name}",
            shell=True,
            capture_output=True,
            text=True,
            timeout=60
        )
        
        if result.returncode == 0:
            print(f"✅ Successfully created Heroku app: {app_name}")
            
            # Extract app URL from output
            output_lines = result.stdout.strip().split('\n')
            app_url = None
            git_url = None
            
            for line in output_lines:
                if 'https://' in line and '.herokuapp.com' in line:
                    if 'git' in line:
                        git_url = line.strip()
                    else:
                        app_url = line.strip()
            
            if not app_url:
                app_url = f"https://{app_name}.herokuapp.com/"
            if not git_url:
                git_url = f"https://git.heroku.com/{app_name}.git"
            
            print(f"   App URL: {app_url}")
            print(f"   Git URL: {git_url}")
            
            return True, {
                'name': app_name,
                'url': app_url,
                'git_url': git_url
            }
        else:
            error_msg = result.stderr.strip()
            if 'Name is already taken' in error_msg:
                print(f"⚠️ App name '{app_name}' is already taken")
                print("Trying with a different name...")
                return create_heroku_app()  # Retry with new name
            else:
                print(f"❌ Failed to create app: {error_msg}")
                return False, None
                
    except subprocess.TimeoutExpired:
        print("⏰ App creation timed out")
        return False, None
    except Exception as e:
        print(f"❌ Error creating app: {e}")
        return False, None

# Ask user for app name preference
print("🏗️ Creating Heroku Application")
print("\nOptions:")
print("1. Auto-generate a unique name")
print("2. Specify your own name")

choice = input("\nChoose option (1 or 2): ").strip()

app_name = None
if choice == "2":
    app_name = input("Enter your desired app name (lowercase, letters/numbers/hyphens only): ").strip().lower()
    if not app_name:
        print("No name provided, using auto-generated name...")
        app_name = None

# Create the app
success, app_info = create_heroku_app(app_name)

if success:
    display(HTML(f"""
    <div style='background-color: #d4edda; padding: 20px; border-radius: 5px; color: #155724; margin: 10px 0;'>
        <strong>🎉 Heroku App Created Successfully!</strong><br><br>
        <strong>App Details:</strong><br>
        • <strong>Name:</strong> <code>{app_info['name']}</code><br>
        • <strong>URL:</strong> <a href="{app_info['url']}" target="_blank">{app_info['url']}</a><br>
        • <strong>Git URL:</strong> <code>{app_info['git_url']}</code><br><br>
        <em>Your app is created but not yet deployed. Continue with the next steps to deploy your Flask application.</em>
    </div>
    """))
    
    # Store app info for later use
    with open('heroku_app_info.txt', 'w') as f:
        f.write(f"APP_NAME={app_info['name']}\n")
        f.write(f"APP_URL={app_info['url']}\n")
        f.write(f"GIT_URL={app_info['git_url']}\n")
    
    print("\n📝 App information saved to heroku_app_info.txt")
    
else:
    display(HTML("""
    <div style='background-color: #f8d7da; padding: 15px; border-radius: 5px; color: #721c24; margin: 10px 0;'>
        <strong>❌ Failed to Create App</strong><br>
        Please check your authentication and try again with a different app name.
    </div>
    """))

print("\n📝 Next steps:")
print("   1. Set up environment variables")
print("   2. Initialize Git repository")
print("   3. Deploy the application")

In [ ]:
#@title ⚙️ Configure Environment Variables
#@markdown Set up necessary environment variables for the Heroku app

import os
import secrets
from IPython.display import HTML, display

def set_heroku_config(app_name, key, value):
    """Set a configuration variable on Heroku"""
    try:
        result = subprocess.run(
            f'heroku config:set {key}="{value}" --app {app_name}',
            shell=True,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            print(f"✅ Set {key}")
            return True
        else:
            print(f"❌ Failed to set {key}: {result.stderr.strip()}")
            return False
            
    except Exception as e:
        print(f"❌ Error setting {key}: {e}")
        return False

def get_app_name():
    """Get app name from saved file or user input"""
    try:
        if os.path.exists('heroku_app_info.txt'):
            with open('heroku_app_info.txt', 'r') as f:
                for line in f:
                    if line.startswith('APP_NAME='):
                        return line.split('=', 1)[1].strip()
    except:
        pass
    
    # If not found, ask user
    return input("Enter your Heroku app name: ").strip()

# Get app name
app_name = get_app_name()

if not app_name:
    print("❌ No app name provided. Please create an app first.")
else:
    print(f"⚙️ Configuring environment variables for app: {app_name}")
    
    # Generate a secure secret key
    secret_key = secrets.token_urlsafe(32)
    
    # Configuration variables to set
    config_vars = {
        'SECRET_KEY': secret_key,
        'FLASK_ENV': 'production',
        'PYTHONPATH': '/app',
    }
    
    print("\n🔧 Setting configuration variables...")
    
    success_count = 0
    total_vars = len(config_vars)
    
    for key, value in config_vars.items():
        if set_heroku_config(app_name, key, value):
            success_count += 1
    
    # Optional: Ask user for additional environment variables
    print("\n🔧 Optional: Add custom environment variables")
    add_custom = input("Do you want to add any custom environment variables? (y/n): ").strip().lower()
    
    if add_custom == 'y':
        while True:
            key = input("\nEnter variable name (or 'done' to finish): ").strip()
            if key.lower() == 'done' or not key:
                break
            
            value = input(f"Enter value for {key}: ").strip()
            if value:
                if set_heroku_config(app_name, key, value):
                    success_count += 1
                    total_vars += 1
    
    # Display current configuration
    print("\n📋 Retrieving current configuration...")
    try:
        result = subprocess.run(
            f"heroku config --app {app_name}",
            shell=True,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            config_output = result.stdout.strip()
            print("\n📋 Current Configuration:")
            print(config_output)
        else:
            print(f"⚠️ Could not retrieve configuration: {result.stderr.strip()}")
            
    except Exception as e:
        print(f"⚠️ Error retrieving configuration: {e}")
    
    # Summary
    if success_count == total_vars:
        display(HTML(f"""
        <div style='background-color: #d4edda; padding: 15px; border-radius: 5px; color: #155724; margin: 10px 0;'>
            <strong>✅ Configuration Complete!</strong><br>
            Successfully set {success_count}/{total_vars} environment variables.<br>
            Your app is now configured for production deployment.
        </div>
        """))
    else:
        display(HTML(f"""
        <div style='background-color: #fff3cd; padding: 15px; border-radius: 5px; color: #856404; margin: 10px 0;'>
            <strong>⚠️ Partial Configuration</strong><br>
            Set {success_count}/{total_vars} environment variables.<br>
            Some variables may not have been set correctly.
        </div>
        """))

print("\n📝 Configuration Tips:")
print("   • SECRET_KEY: Used for session security (auto-generated)")
print("   • FLASK_ENV: Set to 'production' for deployment")
print("   • You can add more variables later using: heroku config:set KEY=VALUE")
print("   • View all config vars: heroku config --app your-app-name")

In [ ]:
#@title 📦 Initialize Git Repository and Deploy
#@markdown Set up Git repository and deploy the application to Heroku

import subprocess
import os
from IPython.display import HTML, display

def run_git_command(command, description="", timeout=60):
    """Run git command with error handling"""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        if result.returncode == 0:
            print(f"✅ {description} completed")
            if result.stdout.strip():
                # Show first few lines of output
                output_lines = result.stdout.strip().split('\n')[:3]
                for line in output_lines:
                    if line.strip():
                        print(f"   {line.strip()}")
            return True, result.stdout
        else:
            print(f"❌ {description} failed")
            if result.stderr.strip():
                print(f"   Error: {result.stderr.strip()[:200]}")
            return False, result.stderr
            
    except subprocess.TimeoutExpired:
        print(f"⏰ {description} timed out")
        return False, "Command timed out"
    except Exception as e:
        print(f"❌ {description} failed: {e}")
        return False, str(e)

def get_app_info():
    """Get app information from saved file"""
    app_info = {}
    try:
        if os.path.exists('heroku_app_info.txt'):
            with open('heroku_app_info.txt', 'r') as f:
                for line in f:
                    if '=' in line:
                        key, value = line.strip().split('=', 1)
                        app_info[key] = value
    except:
        pass
    return app_info

# Get app information
app_info = get_app_info()
app_name = app_info.get('APP_NAME')

if not app_name:
    app_name = input("Enter your Heroku app name: ").strip()

if not app_name:
    print("❌ No app name provided. Please create an app first.")
else:
    print(f"📦 Preparing deployment for app: {app_name}")
    print("This process will:")
    print("   1. Initialize Git repository")
    print("   2. Add Heroku remote")
    print("   3. Commit all files")
    print("   4. Deploy to Heroku")
    print("   5. Verify deployment")
    
    proceed = input("\nProceed with deployment? (y/n): ").strip().lower()
    
    if proceed == 'y':
        print("\n🚀 Starting deployment process...\n")
        
        # Step 1: Initialize Git repository
        success, output = run_git_command(
            "git init",
            "Initializing Git repository"
        )
        
        if success:
            # Step 2: Configure Git (if needed)
            run_git_command(
                'git config user.email "colab@example.com"',
                "Setting Git email"
            )
            run_git_command(
                'git config user.name "Colab User"',
                "Setting Git username"
            )
            
            # Step 3: Add Heroku remote
            success, output = run_git_command(
                f"heroku git:remote -a {app_name}",
                "Adding Heroku remote"
            )
        
        if success:
            # Step 4: Add all files
            success, output = run_git_command(
                "git add .",
                "Adding files to Git"
            )
        
        if success:
            # Step 5: Commit files
            success, output = run_git_command(
                'git commit -m "Initial deployment from Google Colab"',
                "Committing files"
            )
        
        if success:
            # Step 6: Deploy to Heroku
            print("\n🚀 Deploying to Heroku...")
            print("This may take several minutes...")
            
            success, output = run_git_command(
                "git push heroku main",
                "Pushing to Heroku",
                timeout=300  # 5 minutes timeout for deployment
            )
            
            if success:
                print("\n🎉 Deployment completed successfully!")
                
                # Step 7: Ensure at least one dyno is running
                print("\n🔧 Ensuring app is running...")
                subprocess.run(
                    f"heroku ps:scale web=1 --app {app_name}",
                    shell=True,
                    capture_output=True
                )
                
                # Step 8: Get app URL and test
                app_url = app_info.get('APP_URL', f"https://{app_name}.herokuapp.com/")
                
                print(f"\n🌐 Testing deployment at: {app_url}")
                
                # Wait a moment for app to start
                time.sleep(10)
                
                try:
                    import requests
                    response = requests.get(app_url, timeout=30)
                    if response.status_code == 200:
                        print("✅ App is responding successfully!")
                        deployment_success = True
                    else:
                        print(f"⚠️ App responded with status code: {response.status_code}")
                        deployment_success = True  # Still consider it successful
                except Exception as e:
                    print(f"⚠️ Could not test app immediately: {e}")
                    print("This is normal - the app may still be starting up.")
                    deployment_success = True
                
                if deployment_success:
                    display(HTML(f"""
                    <div style='background-color: #d4edda; padding: 20px; border-radius: 10px; color: #155724; margin: 20px 0;'>
                        <h3 style='margin-top: 0; color: #155724;'>🎉 Deployment Successful!</h3>
                        <p><strong>Your Flask application is now live on Heroku!</strong></p>
                        <p><strong>🌐 App URL:</strong> <a href="{app_url}" target="_blank" style='color: #155724; font-weight: bold;'>{app_url}</a></p>
                        <p><strong>📱 App Name:</strong> <code>{app_name}</code></p>
                        <hr style='border-color: #c3e6cb;'>
                        <p><strong>✨ Features Available:</strong></p>
                        <ul>
                            <li>Interactive Calculator</li>
                            <li>File Upload & Data Visualization</li>
                            <li>API Integration Demo</li>
                            <li>Responsive Bootstrap UI</li>
                        </ul>
                        <p><em>🎯 Click the URL above to view your deployed application!</em></p>
                    </div>
                    """))
                    
                    # Save deployment info
                    with open('deployment_success.txt', 'w') as f:
                        f.write("Deployment successful!\n")
                        f.write(f"App Name: {app_name}\n")
                        f.write(f"App URL: {app_url}\n")
                        f.write(f"Deployed at: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                
            else:
                print("❌ Deployment failed")
                display(HTML("""
                <div style='background-color: #f8d7da; padding: 15px; border-radius: 5px; color: #721c24; margin: 10px 0;'>
                    <strong>❌ Deployment Failed</strong><br>
                    Check the error messages above and try again.
                </div>
                """))
    else:
        print("Deployment cancelled by user.")

print("\n📝 Useful Heroku Commands:")
print(f"   • View logs: heroku logs --tail --app {app_name}")
print(f"   • Open app: heroku open --app {app_name}")
print(f"   • Check status: heroku ps --app {app_name}")
print(f"   • Restart app: heroku restart --app {app_name}")

In [ ]:
#@title 🔍 Verify Deployment and Monitor
#@markdown Check deployment status, view logs, and monitor the application

import subprocess
import requests
import time
import os
from IPython.display import HTML, display

def check_heroku_status(app_name):
    """Check Heroku app status"""
    try:
        result = subprocess.run(
            f"heroku ps --app {app_name}",
            shell=True,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            return True, result.stdout.strip()
        else:
            return False, result.stderr.strip()
    except Exception as e:
        return False, str(e)

def get_heroku_logs(app_name, lines=50):
    """Get recent Heroku logs"""
    try:
        result = subprocess.run(
            f"heroku logs --tail --num {lines} --app {app_name}",
            shell=True,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            return True, result.stdout.strip()
        else:
            return False, result.stderr.strip()
    except Exception as e:
        return False, str(e)

def test_app_endpoints(app_url):
    """Test various app endpoints"""
    endpoints = {
        'Home Page': '',
        'Calculator': 'calculator',
        'File Upload': 'upload',
        'API Demo': 'api-demo',
        'Quote API': 'api/quote'
    }
    
    results = {}
    
    for name, endpoint in endpoints.items():
        test_url = f"{app_url.rstrip('/')}/{endpoint}"
        try:
            response = requests.get(test_url, timeout=10)
            results[name] = {
                'status': response.status_code,
                'success': response.status_code == 200,
                'url': test_url
            }
        except Exception as e:
            results[name] = {
                'status': 'Error',
                'success': False,
                'error': str(e),
                'url': test_url
            }
    
    return results

# Get app information
def get_app_info():
    app_info = {}
    try:
        if os.path.exists('heroku_app_info.txt'):
            with open('heroku_app_info.txt', 'r') as f:
                for line in f:
                    if '=' in line:
                        key, value = line.strip().split('=', 1)
                        app_info[key] = value
    except:
        pass
    return app_info

app_info = get_app_info()
app_name = app_info.get('APP_NAME')
app_url = app_info.get('APP_URL')

if not app_name:
    app_name = input("Enter your Heroku app name: ").strip()
    if not app_url:
        app_url = f"https://{app_name}.herokuapp.com/"

if app_name:
    print(f"🔍 Verifying deployment for: {app_name}")
    print(f"🌐 App URL: {app_url}")
    print("\n" + "="*50)
    
    # 1. Check Heroku app status
    print("\n📊 Checking Heroku App Status...")
    success, status_output = check_heroku_status(app_name)
    
    if success:
        print("✅ App status retrieved successfully:")
        print(status_output)
    else:
        print(f"❌ Could not get app status: {status_output}")
    
    # 2. Test app endpoints
    print("\n🧪 Testing Application Endpoints...")
    endpoint_results = test_app_endpoints(app_url)
    
    working_endpoints = 0
    total_endpoints = len(endpoint_results)
    
    for name, result in endpoint_results.items():
        if result['success']:
            print(f"✅ {name}: Status {result['status']}")
            working_endpoints += 1
        else:
            error_info = result.get('error', f"Status {result['status']}")
            print(f"❌ {name}: {error_info}")
    
    # 3. Get recent logs
    print("\n📋 Recent Application Logs...")
    success, logs = get_heroku_logs(app_name, 20)
    
    if success:
        print("Recent logs:")
        print("-" * 40)
        log_lines = logs.split('\n')[-10:]  # Show last 10 lines
        for line in log_lines:
            if line.strip():
                print(line)
        print("-" * 40)
    else:
        print(f"❌ Could not retrieve logs: {logs}")
    
    # 4. Performance test
    print("\n⚡ Performance Test...")
    try:
        start_time = time.time()
        response = requests.get(app_url, timeout=15)
        response_time = time.time() - start_time
        
        print(f"Response time: {response_time:.2f} seconds")
        print(f"Status code: {response.status_code}")
        print(f"Content length: {len(response.content)} bytes")
        
    except Exception as e:
        print(f"❌ Performance test failed: {e}")
    
    # Summary
    print("\n" + "="*50)
    print("📊 DEPLOYMENT SUMMARY")
    print("="*50)
    
    if working_endpoints == total_endpoints:
        status_color = "#d4edda"
        status_text_color = "#155724"
        status_icon = "✅"
        status_message = "All systems operational!"
    elif working_endpoints > 0:
        status_color = "#fff3cd"
        status_text_color = "#856404"
        status_icon = "⚠️"
        status_message = "Partially operational"
    else:
        status_color = "#f8d7da"
        status_text_color = "#721c24"
        status_icon = "❌"
        status_message = "Issues detected"
    
    display(HTML(f"""
    <div style='background-color: {status_color}; padding: 20px; border-radius: 10px; color: {status_text_color}; margin: 20px 0;'>
        <h3 style='margin-top: 0; color: {status_text_color};'>{status_icon} Deployment Status: {status_message}</h3>
        
        <div style='margin: 15px 0;'>
            <strong>📱 App Details:</strong><br>
            • <strong>Name:</strong> <code>{app_name}</code><br>
            • <strong>URL:</strong> <a href="{app_url}" target="_blank" style='color: {status_text_color}; font-weight: bold;'>{app_url}</a><br>
            • <strong>Endpoints Working:</strong> {working_endpoints}/{total_endpoints}
        </div>
        
        <div style='margin: 15px 0;'>
            <strong>🛠️ Management Commands:</strong><br>
            • View logs: <code>heroku logs --tail --app {app_name}</code><br>
            • Restart app: <code>heroku restart --app {app_name}</code><br>
            • Open app: <code>heroku open --app {app_name}</code><br>
            • Check status: <code>heroku ps --app {app_name}</code>
        </div>
        
        <p style='margin-bottom: 0;'><strong>🎉 Congratulations! Your Flask application has been successfully deployed from Google Colab to Heroku!</strong></p>
    </div>
    """))
    
    # Save verification results
    with open('verification_results.txt', 'w') as f:
        f.write(f"Verification completed at: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"App Name: {app_name}\n")
        f.write(f"App URL: {app_url}\n")
        f.write(f"Working Endpoints: {working_endpoints}/{total_endpoints}\n")
        f.write(f"Status: {status_message}\n")

else:
    print("❌ No app information found. Please create and deploy an app first.")

print("\n🎯 Next Steps:")
print("   • Test all application features")
print("   • Monitor application logs")
print("   • Set up custom domain (optional)")
print("   • Configure SSL certificate (optional)")
print("   • Set up monitoring and alerts")

---

# 🎉 Deployment Complete!

## 📚 What You've Accomplished

Congratulations! You've successfully:

✅ **Built a Complete Flask Application** with:
- Interactive calculator with multiple operations
- File upload and CSV data visualization
- API integration demos
- Responsive Bootstrap UI
- Error handling and validation

✅ **Deployed to Heroku** with:
- Proper configuration files (Procfile, requirements.txt, runtime.txt)
- Environment variables setup
- Git repository initialization
- Automated deployment process

✅ **Verified Deployment** with:
- Status monitoring
- Endpoint testing
- Log analysis
- Performance testing

## 🛠️ Troubleshooting Tips

If you encounter issues:

1. **App not responding**: Check logs with `heroku logs --tail --app your-app-name`
2. **Build failures**: Verify requirements.txt and Python version in runtime.txt
3. **Environment issues**: Check config vars with `heroku config --app your-app-name`
4. **Performance issues**: Consider upgrading to a paid Heroku plan

## 🚀 Next Steps

- **Custom Domain**: Set up a custom domain for your app
- **Database**: Add PostgreSQL for data persistence
- **Monitoring**: Set up application monitoring and alerts
- **CI/CD**: Implement continuous integration and deployment
- **Scaling**: Configure auto-scaling based on traffic

## 📖 Additional Resources

- [Heroku Dev Center](https://devcenter.heroku.com/)
- [Flask Documentation](https://flask.palletsprojects.com/)
- [Bootstrap Documentation](https://getbootstrap.com/docs/)
- [Plotly Documentation](https://plotly.com/python/)

---

**🎯 You now have a complete understanding of deploying web applications from Google Colab to Heroku!**